In [ ]:
import numpy as np
import math
import copy
from pprint import pprint
from collections import defaultdict

In [ ]:
class MonitoringStation:
    def __init__(self):
        self.x = 0  # distance from left edge
        self.y = 0  # distance from top edge
        self.map = None
    
    def open_file(self, path_file):
        self.map = []
        with open(path_file, "r") as f:
            for line in f:
                self.map.append(list(line.strip()))
    
    @property
    def value_map(self):
        value_map = copy.deepcopy(self.map)
        for key, val in self.compute_value_map().items():
            value_map[key[0]][key[1]] = str(len(val))
        
        pprint(value_map)
    
    def get_coord(self):
        coord_x, coord_y = np.where(np.asarray(self.map) == '#')
        return list(zip(coord_x, coord_y))
    
    @staticmethod
    def vec(ast1, ast2):
        return (ast2[0] - ast1[0], ast2[1] - ast1[1])
    
    @staticmethod
    def unit_vector(vector):
            return vector / np.linalg.norm(vector)
    
    def angle(self, v1, v2):
        v1_u = self.unit_vector(v1)
        v2_u = self.unit_vector(v2)
        return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
    
    def compute_value_map(self):
        asteroid_map = self.get_coord()
        asteroid_mapping = defaultdict(set)
        
        for ast_base in asteroid_map:
            for ast_new in asteroid_map:
                if ast_base != ast_new:
                    v = tuple(np.round(self.unit_vector(self.vec(ast_base, ast_new)), decimals=10))
                    asteroid_mapping[ast_base].add(v)

        return asteroid_mapping
    
    def part_1(self):
        d = self.compute_value_map()
        key = max(d, key=lambda k: len(d[k]))
        return key, len(d[key])
    
    def part_2(self, ast_base):
        v_up = self.vec(ast_base, (19, 31))
        
        asteroid_map = self.get_coord()
        asteroid_mapping = defaultdict(list)
        
        for ast in asteroid_map:
            if ast != ast_base:
                v_ast = self.vec(ast_base, ast)
                angle = self.angle(v_up, v_ast)
                if ast_base[1] > ast[1]:
                    angle = 2*math.pi - angle
                asteroid_mapping[angle].append(ast)
            
        for l in asteroid_mapping.values():
            l.sort(key=lambda ast: np.linalg.norm(self.vec(ast_base, ast)))
                
        nb_boom = 0
        while(nb_boom < 200):
            for angle in sorted(asteroid_mapping.keys()):
                if asteroid_mapping[angle]:
                    ast = asteroid_mapping[angle].pop(0)
                    nb_boom +=1
                    if nb_boom == 200:
                        return ast
            else:
                if all(val == [] for val in asteroid_mapping.values()):
                    break
                


In [ ]:
ms = MonitoringStation()
ms.open_file("/home/bchatillon/Documents/Advent-of-Code/2019/10_data.txt")

In [ ]:
ms.map

In [ ]:
ms.value_map

In [ ]:
ast_base, dist = ms.part_1()
dist

In [ ]:
y, x = ms.part_2(ast_base)
print((x*100 + y))